In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
moaks_sq01 = pd.read_csv('/home/anastasis/EMC_Thesis/OAI Data/OAICompleteData_ASCII/kMRI_SQ_MOAKS_BICL01.txt', sep="|")

In [3]:
print(moaks_sq01.info(),'\n',moaks_sq01.iloc[:,-3])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086 entries, 0 to 3085
Columns: 121 entries, ID to V01MTCMNTS
dtypes: float64(116), int64(2), object(3)
memory usage: 2.8+ MB
None 
 0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3081   NaN
3082   NaN
3083   NaN
3084   NaN
3085   NaN
Name: V01MITBSIG, Length: 3086, dtype: float64


# NOW WE DELETE ALL THE COMMENTS OF THE INITIAL DATAFRAME AND CONVERT THE VALUES TO NUMBERS

In [4]:
# Firstly we replace all the comments
moaks_without_commments = moaks_sq01.replace(r'(\:).*$',r'\1',regex=True)
# Secondly we replace all the '.:' and the ':' bullets with space and remove 'R' from Reader Column
moaks_without_commments_bullets = moaks_without_commments.replace({'\.\:':'',':':''},regex=True)
# Thirdly we remove the Comments and Technical Considerations Columns from the whole Dataset
moaks_without_commments_bullets = moaks_without_commments_bullets.drop(columns=['V01MCMNTS','V01MTCMNTS'])
print(moaks_without_commments_bullets.READPRJ.value_counts())

65     1011
61      852
22      600
63E     299
63A     107
63C      62
63F      60
63B      57
63D      38
Name: READPRJ, dtype: int64


In [5]:
# WE KEEP ONLY THE SUBJECTS THAT ARE PRESENT IN PROJECT 65 AND NOT IN PROJECT 22

moaks_without_commments_bullets_prj22 = moaks_without_commments_bullets.loc[moaks_without_commments_bullets['READPRJ']=='22']
print(moaks_without_commments_bullets_prj22.shape[0])

moaks_without_commments_bullets_prj63A = moaks_without_commments_bullets.loc[moaks_without_commments_bullets['READPRJ']=='63A']
print('Number of subjects with OA at 12 months prj63: {}'.format(moaks_without_commments_bullets_prj63A.shape[0]))
moaks_without_commments_bullets_prj63A_unique_ID = pd.merge(moaks_without_commments_bullets_prj63A,moaks_without_commments_bullets_prj22,on=['ID','SIDE'],how='outer',indicator=True)
moaks_without_commments_bullets_prj63A_unique_ID = moaks_without_commments_bullets_prj63A_unique_ID.loc[(moaks_without_commments_bullets_prj63A_unique_ID._merge=='left_only'),['ID','SIDE']]
moaks_without_commments_bullets_prj63A_unique_info = moaks_without_commments_bullets_prj63A.merge(moaks_without_commments_bullets_prj63A_unique_ID,on=['ID','SIDE'],how='inner')
moaks_without_commments_bullets_prj63A_unique_info = moaks_without_commments_bullets_prj63A_unique_info.drop_duplicates(subset=['ID','SIDE'])
print('Number of subjects with OA at 12 months unique to prj63: {}'.format(moaks_without_commments_bullets_prj63A_unique_info.shape[0]))



600
Number of subjects with OA at 12 months prj63: 107
Number of subjects with OA at 12 months unique to prj63: 97


In [6]:
abc = (pd.merge(moaks_without_commments_bullets_prj63A,moaks_without_commments_bullets_prj22,on=['ID','SIDE'],how='outer',indicator=True))._merge
abc.value_counts()

right_only    590
left_only      97
both           10
Name: _merge, dtype: int64

In [7]:
# THEN WE CONVERT ALL THE DF VALUES TO FLOAT AND THE ID AND SIDE TO INT
moaks_without_commments_bullets_prj63A_unique_info.replace(to_replace={'63A':'63.1'},inplace=True)
moaks_without_c_b_numeric_63A =  moaks_without_commments_bullets_prj63A_unique_info.apply(pd.to_numeric,downcast='float')
moaks_without_c_b_numeric_63A[['ID','SIDE']] = moaks_without_c_b_numeric_63A[['ID','SIDE']].astype('int')
print(moaks_without_c_b_numeric_63A.info())
print('Number of NaN values are : ',moaks_without_c_b_numeric_63A.isna().sum().sum())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Columns: 119 entries, ID to V01MITBSIG
dtypes: float32(117), int64(2)
memory usage: 46.6 KB
None
Number of NaN values are :  7372


In [8]:
print('Number of NaN values in the initial dataframe: {}'.format(moaks_without_c_b_numeric_63A.loc[:,:].isna().sum().to_list()))

Number of NaN values in the initial dataframe: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 0, 0, 0, 0, 0, 0, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 0, 97, 97, 0, 0, 0, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97]


In [9]:
moaks_without_c_b_nan_numeric = moaks_without_c_b_numeric_63A.fillna(-1.0)
moaks_without_c_b_nan_numeric.info()
print('Number of NaN values in the Dataframe: {}'.format(moaks_without_c_b_nan_numeric.isna().sum().to_list()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Columns: 119 entries, ID to V01MITBSIG
dtypes: float32(117), int64(2)
memory usage: 46.6 KB
Number of NaN values in the Dataframe: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
# So the clean version of the moaks_sq01 is the below: 
moaks_grades = moaks_without_c_b_nan_numeric
print(moaks_grades.isna().sum().to_list())
moaks_grades.to_csv('moaks_sq01_initial_numeric_df.csv')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# NOW WE LOAD THE RADIOGRAPH INFORMATION DATABASE IN ORDER TO ACQUIRE THE XR OSTEOPHYTES


In [11]:
kxr_sq_bu01= pd.read_csv('/home/anastasis/EMC_Thesis/OAI Data/OAICompleteData_ASCII/kxr_sq_bu01.txt', sep="|")
KL_prj15 = kxr_sq_bu01[kxr_sq_bu01['readprj'] == 15].copy()
KL_prj15_osteophytes = KL_prj15.loc[:,['ID','SIDE','V01XROSFL','V01XROSFM','V01XROSTL','V01XROSTM','V01XRKL']]

#now we create the df that contains both MOAKS grades and XR Osteophytes
common_sq_kxr_01 = pd.merge(moaks_grades,KL_prj15_osteophytes,on=['ID','SIDE'],how='inner')
common_sq_kxr_01.fillna(0.0,inplace=True)
common_sq_kxr_01

,ID,SIDE,READPRJ,VERSION,V01MCMPM,V01MCMPL,V01MCMFMA,V01MCMFLA,V01MCMFMP,V01MCMFLP,...,V01MPPBUR,V01MSYIC,V01MEFFWK,V01MPOPCYS,V01MITBSIG,V01XROSFL,V01XROSFM,V01XROSTL,V01XROSTM,V01XRKL
0,9002316,2,63.099998,1.2,2.0,1.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,2.0
1,9026695,1,63.099998,1.2,2.0,2.0,0.0,0.0,2.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,3.0
2,9044788,2,63.099998,1.2,1.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1.0,0.0,2.0
3,9062483,2,63.099998,1.2,0.0,0.0,0.0,0.0,2.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,2.0
4,9063928,2,63.099998,1.2,1.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,9943227,2,63.099998,1.2,2.0,2.0,0.0,0.0,2.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,2.0
93,9957519,2,63.099998,1.2,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1.0,1.0,2.0
94,9961728,2,63.099998,1.2,2.0,2.0,1.0,0.0,2.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,2.0
95,9984706,1,63.099998,1.2,0.0,2.0,0.0,2.0,2.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,1.0,1.0,2.0


# NOW WE FIND THE SUBJECTS THAT FULFILL THE OA FEATURES FOR THE WHOLE KNEE JOINT

In [12]:
# FRIST WE CONSTRUCT A NEW DATAFRAME WITH ALL THE OA FEATURES AS COLUMNS
# The 'AnyTCL' column is an assistive variable that indicates as 1.0
# the subjects with at least on variable of thickness cartilage loss >=1.0
# and as 0.0 the subjects that DO NOT have any thickness cartilage loss

moaks_OA_features = pd.DataFrame(
    columns=['ID','SIDE','XR_Osteophytes','FullTCL','AnyTCL',
             'PartialTCL','BML', 'Meniscus_Degradation'])
# the ID and SIDE columns are integers
moaks_OA_features[['ID','SIDE']] = common_sq_kxr_01[['ID','SIDE']]
# and we fill the rest of the df with np.nan values
moaks_OA_features.iloc[:,2:] = np.nan

print('Type of ID and SIDE columns {} and type of the rest of the columns {}'
    .format(type(moaks_OA_features.iloc[0,0]),type(moaks_OA_features.iloc[0,2])))


Type of ID and SIDE columns <class 'numpy.int64'> and type of the rest of the columns <class 'numpy.float64'>


In [13]:
moaks_OA_features

,ID,SIDE,XR_Osteophytes,FullTCL,AnyTCL,PartialTCL,BML,Meniscus_Degradation
0,9002316,2,NaN,NaN,NaN,NaN,NaN,NaN
1,9026695,1,NaN,NaN,NaN,NaN,NaN,NaN
2,9044788,2,NaN,NaN,NaN,NaN,NaN,NaN
3,9062483,2,NaN,NaN,NaN,NaN,NaN,NaN
4,9063928,2,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
92,9943227,2,NaN,NaN,NaN,NaN,NaN,NaN
93,9957519,2,NaN,NaN,NaN,NaN,NaN,NaN
94,9961728,2,NaN,NaN,NaN,NaN,NaN,NaN
95,9984706,1,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Then we fill the FullTCL column with all the subjects with 
# at least one cartilage thickness loss variable larger than 3.0
# and the FullTCL cell value is 1.0
moaks_OA_features.loc[((common_sq_kxr_01['V01MCMFLA'] >= 3.0) | (common_sq_kxr_01['V01MCMFLP'] >= 3.0) |
                (common_sq_kxr_01['V01MCMFLC'] >= 3.0) | (common_sq_kxr_01['V01MCMTLA'] >= 3.0) |
                (common_sq_kxr_01['V01MCMTLP'] >= 3.0) | (common_sq_kxr_01['V01MCMTLC'] >= 3.0) |
                (common_sq_kxr_01['V01MCMFMA'] >= 3.0) | (common_sq_kxr_01['V01MCMFMP'] >= 3.0) |
                (common_sq_kxr_01['V01MCMFMC'] >= 3.0) | (common_sq_kxr_01['V01MCMTMA'] >= 3.0) |
                (common_sq_kxr_01['V01MCMTMP'] >= 3.0) | (common_sq_kxr_01['V01MCMTMC'] >= 3.0)),'FullTCL']=1.0

In [15]:
print('Number of subjects WITH Full CTL : {}'.format((moaks_OA_features['FullTCL']==1.0).sum()))


Number of subjects WITH Full CTL : 6


In [16]:
# Then we fill the rest of the values of Full_CTL column with 0.0 values instead of NaN
moaks_OA_features.loc[((common_sq_kxr_01['V01MCMFLA'] < 3.0) &  (common_sq_kxr_01['V01MCMFLA'] >= 0.0) &
                (common_sq_kxr_01['V01MCMFLP'] < 3.0) & (common_sq_kxr_01['V01MCMFLP'] >= 0.0) &
                (common_sq_kxr_01['V01MCMFLC'] < 3.0) & (common_sq_kxr_01['V01MCMFLC'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMTLA'] < 3.0) & (common_sq_kxr_01['V01MCMTLA'] >= 0.0) &
                (common_sq_kxr_01['V01MCMTLP'] < 3.0) & (common_sq_kxr_01['V01MCMTLP'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMTLC'] < 3.0) & (common_sq_kxr_01['V01MCMTLC'] >= 0.0) &
                (common_sq_kxr_01['V01MCMFMA'] < 3.0) & (common_sq_kxr_01['V01MCMFMA'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMFMP'] < 3.0) & (common_sq_kxr_01['V01MCMFMP'] >= 0.0) &
                (common_sq_kxr_01['V01MCMFMC'] < 3.0) & (common_sq_kxr_01['V01MCMFMC'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMTMA'] < 3.0) & (common_sq_kxr_01['V01MCMTMA'] >= 0.0) &
                (common_sq_kxr_01['V01MCMTMP'] < 3.0) & (common_sq_kxr_01['V01MCMTMP'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMTMC'] < 3.0) & (common_sq_kxr_01['V01MCMTMC'] >= 0.0)),'FullTCL']=0.0
print(moaks_OA_features['FullTCL'].value_counts())


0.0    91
1.0     6
Name: FullTCL, dtype: int64


In [17]:
# now the column AnyTCL is filled based on the initial Thickness Cartilage Loss variables
# if at least one variable value is larger than 1.0 then it has either Full or Partial Thickness Cartilage Loss
# and the AnyTCL cell value is 1.0

moaks_OA_features.loc[((common_sq_kxr_01['V01MCMFLA'] >= 1.0) | (common_sq_kxr_01['V01MCMFLP'] >= 1.0) |
                (common_sq_kxr_01['V01MCMFLC'] >= 1.0) | (common_sq_kxr_01['V01MCMTLA'] >= 1.0) |
                (common_sq_kxr_01['V01MCMTLP'] >= 1.0) | (common_sq_kxr_01['V01MCMTLC'] >= 1.0) |
                (common_sq_kxr_01['V01MCMFMA'] >= 1.0) | (common_sq_kxr_01['V01MCMFMP'] >= 1.0) |
                (common_sq_kxr_01['V01MCMFMC'] >= 1.0) | (common_sq_kxr_01['V01MCMTMA'] >= 1.0) |
                (common_sq_kxr_01['V01MCMTMP'] >= 1.0) | (common_sq_kxr_01['V01MCMTMC'] >= 1.0)),'AnyTCL']=1.0



In [18]:
# if all Thickness Cartilage Loss variables are less than 1.0 then the subject has no Thickness Cartilage Loss
# and the AnyTCL cell value is 0.0
moaks_OA_features.loc[((common_sq_kxr_01['V01MCMFLA'] < 1.0) & (common_sq_kxr_01['V01MCMFLA'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMFLP'] < 1.0) & (common_sq_kxr_01['V01MCMFLP'] >= 0.0) &
                (common_sq_kxr_01['V01MCMFLC'] < 1.0) & (common_sq_kxr_01['V01MCMFLC'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMTLA'] < 1.0) & (common_sq_kxr_01['V01MCMTLA'] >= 0.0) &
                (common_sq_kxr_01['V01MCMTLP'] < 1.0) & (common_sq_kxr_01['V01MCMTLP'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMTLC'] < 1.0) & (common_sq_kxr_01['V01MCMTLC'] >= 0.0) &
                (common_sq_kxr_01['V01MCMFMA'] < 1.0) & (common_sq_kxr_01['V01MCMFMA'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMFMP'] < 1.0) & (common_sq_kxr_01['V01MCMFMP'] >= 0.0) &
                (common_sq_kxr_01['V01MCMFMC'] < 1.0) & (common_sq_kxr_01['V01MCMFMC'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMTMA'] < 1.0) & (common_sq_kxr_01['V01MCMTMA'] >= 0.0) &
                (common_sq_kxr_01['V01MCMTMP'] < 1.0) & (common_sq_kxr_01['V01MCMTMP'] >= 0.0) & 
                (common_sq_kxr_01['V01MCMTMC'] < 1.0) & (common_sq_kxr_01['V01MCMTMC'] >= 0.0)),'AnyTCL']=0.0
print('Number of Subjects WITHOUT Any Cartilage Thickness Loss: {}'.
        format((moaks_OA_features['AnyTCL']==0.0).sum()))


Number of Subjects WITHOUT Any Cartilage Thickness Loss: 9


In [19]:
print(moaks_OA_features.AnyTCL.value_counts())

1.0    88
0.0     9
Name: AnyTCL, dtype: int64


In [20]:
# Now we fill the PartialTCL column initially with the values of the AnyTCL column
# Where the FullTCL column is 1.0 then the PartialTCL is 0.0
# So if the PartialTCL is 1.0 then the Thickness Cartilage Loss variables are between 1.0 and 3.0

moaks_OA_features['PartialTCL'] = moaks_OA_features['AnyTCL']
moaks_OA_features.loc[(moaks_OA_features['FullTCL']==1),'PartialTCL']=0.0
print('Number of Subjects with Partial Thickness Cartilage Loss: {}'.
        format((moaks_OA_features['PartialTCL']==1.0).sum()))

Number of Subjects with Partial Thickness Cartilage Loss: 82


In [21]:
(common_sq_kxr_01.loc[:,['V01MCMFLA','V01MCMFLP','V01MCMFLP','V01MCMTLA','V01MCMTLP','V01MCMTLC'
                    ,'V01MCMFMA','V01MCMFMP','V01MCMFMP','V01MCMTMA','V01MCMTMP','V01MCMTMC']]==-1.0).sum()

V01MCMFLA    0
V01MCMFLP    0
V01MCMFLP    0
V01MCMTLA    0
V01MCMTLP    0
V01MCMTLC    0
V01MCMFMA    0
V01MCMFMP    0
V01MCMFMP    0
V01MCMTMA    0
V01MCMTMP    0
V01MCMTMC    0
dtype: int64

In [22]:
# NOW WE FILL THE XR OSTEOPHYTES COLUMN

moaks_OA_features.loc[((common_sq_kxr_01['V01XROSFL'] >= 1.0) | 
                (common_sq_kxr_01['V01XROSFM'] >= 1.0) | 
                (common_sq_kxr_01['V01XROSTL'] >= 1.0) | 
                (common_sq_kxr_01['V01XROSTM'] >= 1.0) ),'XR_Osteophytes'] = 1.0

moaks_OA_features.loc[((common_sq_kxr_01['V01XROSFL'] < 1.0) & (common_sq_kxr_01['V01XROSFL'] >= 0.0) & 
                (common_sq_kxr_01['V01XROSFM'] < 1.0) & (common_sq_kxr_01['V01XROSFM'] >= 0.0) & 
                (common_sq_kxr_01['V01XROSTL'] < 1.0) & (common_sq_kxr_01['V01XROSTL'] >= 0.0) & 
                (common_sq_kxr_01['V01XROSTM'] < 1.0) & (common_sq_kxr_01['V01XROSTM'] >= 0.0)),'XR_Osteophytes'] = 0.0

print('With XR Osteophytes->1.0, Without->0.0 \n{}'.format(moaks_OA_features.XR_Osteophytes.value_counts(dropna=False)))


With XR Osteophytes->1.0, Without->0.0 
1.0    97
Name: XR_Osteophytes, dtype: int64


In [23]:
(common_sq_kxr_01.loc[:,['V01XROSFL','V01XROSFM','V01XROSTL','V01XROSTM']]==-1.0).sum()

V01XROSFL    0
V01XROSFM    0
V01XROSTL    0
V01XROSTM    0
dtype: int64

In [24]:
# NOW WE FILL THE BML COLUMN

moaks_OA_features.loc[((common_sq_kxr_01['V01MBMSFLC'] >= 1.0) | (common_sq_kxr_01['V01MBMSFLP'] >= 1.0) |
                (common_sq_kxr_01['V01MBMSTLA'] >= 1.0) | (common_sq_kxr_01['V01MBMSTLC'] >= 1.0) |
                (common_sq_kxr_01['V01MBMSTLP'] >= 1.0) | (common_sq_kxr_01['V01MBMSFMC'] >= 1.0) | 
                (common_sq_kxr_01['V01MBMSFMP'] >= 1.0) | (common_sq_kxr_01['V01MBMSTMA'] >= 1.0) | 
                (common_sq_kxr_01['V01MBMSTMC'] >= 1.0) | (common_sq_kxr_01['V01MBMSTMP'] >= 1.0)),'BML'] = 1.0

moaks_OA_features.loc[((common_sq_kxr_01['V01MBMSFLC'] < 1.0) & (common_sq_kxr_01['V01MBMSFLC'] >= 0.0) &
                (common_sq_kxr_01['V01MBMSFLP'] < 1.0) & (common_sq_kxr_01['V01MBMSFLP'] >= 0.0) &
                (common_sq_kxr_01['V01MBMSTLA'] < 1.0) & (common_sq_kxr_01['V01MBMSTLA'] >= 0.0) &
                (common_sq_kxr_01['V01MBMSTLC'] < 1.0) & (common_sq_kxr_01['V01MBMSTLC'] >= 0.0) &
                (common_sq_kxr_01['V01MBMSTLP'] < 1.0) & (common_sq_kxr_01['V01MBMSTLP'] >= 0.0) & 
                (common_sq_kxr_01['V01MBMSFMC'] < 1.0) & (common_sq_kxr_01['V01MBMSFMC'] >= 0.0) & 
                (common_sq_kxr_01['V01MBMSFMP'] < 1.0) & (common_sq_kxr_01['V01MBMSFMP'] >= 0.0) & 
                (common_sq_kxr_01['V01MBMSTMA'] < 1.0) & (common_sq_kxr_01['V01MBMSTMA'] >= 0.0) & 
                (common_sq_kxr_01['V01MBMSTMC'] < 1.0) & (common_sq_kxr_01['V01MBMSTMC'] >= 0.0) & 
                (common_sq_kxr_01['V01MBMSTMP'] < 1.0) & (common_sq_kxr_01['V01MBMSTMP'] >= 0.0)),'BML'] = 0.0
print('With BML->1.0, Without->0.0 \n{}'.format(moaks_OA_features.BML.value_counts(dropna=False)))


With BML->1.0, Without->0.0 
1.0    50
0.0    43
NaN     4
Name: BML, dtype: int64


In [25]:
(common_sq_kxr_01.loc[:,['V01MBMSFLC','V01MBMSFLP','V01MBMSTLA','V01MBMSTLC','V01MBMSTLP',
                    'V01MBMSFMC','V01MBMSFMP','V01MBMSTMA','V01MBMSTMC','V01MBMSTMP']]==-1.0).sum()

V01MBMSFLC    0
V01MBMSFLP    0
V01MBMSTLA    0
V01MBMSTLC    0
V01MBMSTLP    0
V01MBMSFMC    2
V01MBMSFMP    4
V01MBMSTMA    0
V01MBMSTMC    1
V01MBMSTMP    0
dtype: int64

In [26]:
# AND NOW WE FILL THE MENISCUS DEGRADATION COLUMN

moaks_OA_features.loc[((common_sq_kxr_01['V01MMTLA'] >= 2.0) | (common_sq_kxr_01['V01MMTLB'] >= 2.0) |
                (common_sq_kxr_01['V01MMTLP'] >= 2.0) | (common_sq_kxr_01['V01MMRTL'] >= 2.0) |
                (common_sq_kxr_01['V01MMTMA'] >= 2.0) | (common_sq_kxr_01['V01MMTMB'] >= 2.0) |
                (common_sq_kxr_01['V01MMTMP'] >= 2.0) | (common_sq_kxr_01['V01MMRTM'] >= 2.0) |
                (common_sq_kxr_01['V01MMXLL'] >= 1.0) | (common_sq_kxr_01['V01MMXLA'] >= 1.0) |
                (common_sq_kxr_01['V01MMXMM'] >= 1.0) | (common_sq_kxr_01['V01MMXMA'] >= 1.0)),'Meniscus_Degradation'] = 1.0

moaks_OA_features.loc[((common_sq_kxr_01['V01MMTLA'] < 2.0) & (common_sq_kxr_01['V01MMTLA'] >= 0.0) & 
                (common_sq_kxr_01['V01MMTLB'] < 2.0) & (common_sq_kxr_01['V01MMTLB'] >= 0.0) &
                (common_sq_kxr_01['V01MMTLP'] < 2.0) & (common_sq_kxr_01['V01MMTLP'] >= 0.0) & 
                (common_sq_kxr_01['V01MMRTL'] < 2.0) & (common_sq_kxr_01['V01MMRTL'] >= 0.0) &
                (common_sq_kxr_01['V01MMTMA'] < 2.0) & (common_sq_kxr_01['V01MMTMA'] >= 0.0) & 
                (common_sq_kxr_01['V01MMTMB'] < 2.0) & (common_sq_kxr_01['V01MMTMB'] >= 0.0) &
                (common_sq_kxr_01['V01MMTMP'] < 2.0) & (common_sq_kxr_01['V01MMTMP'] >= 0.0) & 
                (common_sq_kxr_01['V01MMRTM'] < 2.0) & (common_sq_kxr_01['V01MMRTM'] >= 0.0) &
                (common_sq_kxr_01['V01MMXLL'] < 1.0) & (common_sq_kxr_01['V01MMXLL'] >= 0.0) & 
                (common_sq_kxr_01['V01MMXLA'] < 1.0) & (common_sq_kxr_01['V01MMXLA'] >= 0.0) &
                (common_sq_kxr_01['V01MMXMM'] < 1.0) & (common_sq_kxr_01['V01MMXMM'] >= 0.0) & 
                (common_sq_kxr_01['V01MMXMA'] < 1.0) & (common_sq_kxr_01['V01MMXMA'] >= 0.0)),'Meniscus_Degradation'] = 0.0

print('With Meniscus Degradation ->1.0, Without->0.0 \n{}'.format(moaks_OA_features.Meniscus_Degradation.value_counts(dropna=False)))

# print('Number of Subjects WITH Meniscus Degradation: {}'.
#         format((moaks_OA_features['Meniscus_Degradation']==1.0).sum()))

# print('Number of Subjects WITHOUT Meniscus Degradation: {}'.
#         format((moaks_OA_features['Meniscus_Degradation']==0.0).sum()))

With Meniscus Degradation ->1.0, Without->0.0 
1.0    76
NaN    21
Name: Meniscus_Degradation, dtype: int64


In [27]:
(common_sq_kxr_01.loc[:,['V01MMTLA','V01MMTLB','V01MMTLP','V01MMRTL','V01MMTMA','V01MMTMB','V01MMTMP','V01MMRTM',
            'V01MMXLL','V01MMXLA','V01MMXMM','V01MMXMA']]==-1.0).sum()

V01MMTLA     0
V01MMTLB     0
V01MMTLP     0
V01MMRTL     0
V01MMTMA     0
V01MMTMB     0
V01MMTMP     0
V01MMRTM     0
V01MMXLL     0
V01MMXLA    97
V01MMXMM     0
V01MMXMA    97
dtype: int64

In [28]:
print('Number of NaN values per column: {}'.format(moaks_OA_features.isna().sum().to_list()))

Number of NaN values per column: [0, 0, 0, 0, 0, 0, 4, 21]


In [29]:
moaks_OA_features.fillna(-1.0,inplace=True)
moaks_OA_features

,ID,SIDE,XR_Osteophytes,FullTCL,AnyTCL,PartialTCL,BML,Meniscus_Degradation
0,9002316,2,1.0,0.0,1.0,1.0,1.0,1.0
1,9026695,1,1.0,0.0,1.0,1.0,1.0,1.0
2,9044788,2,1.0,0.0,1.0,1.0,1.0,-1.0
3,9062483,2,1.0,0.0,1.0,1.0,0.0,-1.0
4,9063928,2,1.0,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
92,9943227,2,1.0,0.0,1.0,1.0,1.0,1.0
93,9957519,2,1.0,0.0,1.0,1.0,0.0,1.0
94,9961728,2,1.0,0.0,1.0,1.0,0.0,-1.0
95,9984706,1,1.0,0.0,1.0,1.0,1.0,1.0


In [30]:
moaks_OA_features.to_csv('moaks_sq01_OA_features.csv')

# NOW WE CONSTRUCT THE DF THAT CONTAINS THE OA CRITERIA AS COLUMNS

In [31]:
moaks_OA_criteria = pd.DataFrame(columns=['ID','SIDE',
             'Osteo_AND_FullTCL',
             'Osteo_AND_PartialTCL_AND_BML',
             'Osteo_AND_PartialTCL_AND_Meniscus_Degradation',
             'Osteo_AND_BML_AND_Meniscus_Degradation',
             'FullTCL_AND_BML_AND_Meniscus_Degradation'])
# the ID and SIDE columns are integers
moaks_OA_criteria[['ID','SIDE']] = common_sq_kxr_01[['ID','SIDE']]
# and we fill the rest of the df with np.nan values
moaks_OA_criteria.iloc[:,2:] = np.nan
moaks_OA_criteria

,ID,SIDE,Osteo_AND_FullTCL,Osteo_AND_PartialTCL_AND_BML,Osteo_AND_PartialTCL_AND_Meniscus_Degradation,Osteo_AND_BML_AND_Meniscus_Degradation,FullTCL_AND_BML_AND_Meniscus_Degradation
0,9002316,2,NaN,NaN,NaN,NaN,NaN
1,9026695,1,NaN,NaN,NaN,NaN,NaN
2,9044788,2,NaN,NaN,NaN,NaN,NaN
3,9062483,2,NaN,NaN,NaN,NaN,NaN
4,9063928,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
92,9943227,2,NaN,NaN,NaN,NaN,NaN
93,9957519,2,NaN,NaN,NaN,NaN,NaN
94,9961728,2,NaN,NaN,NaN,NaN,NaN
95,9984706,1,NaN,NaN,NaN,NaN,NaN


In [32]:
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] == 1.0) & 
            (moaks_OA_features['FullTCL'] == 1.0)),'Osteo_AND_FullTCL'] = 1.0
         
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] != 1.0) | 
            (moaks_OA_features['FullTCL'] != 1.0)),'Osteo_AND_FullTCL'] = 0.0

print('With Definite Osteophytes AND FullTCL->1.0, Without->0.0: \n{}'.
        format(moaks_OA_criteria['Osteo_AND_FullTCL'].value_counts(dropna=False)))        

With Definite Osteophytes AND FullTCL->1.0, Without->0.0: 
0.0    91
1.0     6
Name: Osteo_AND_FullTCL, dtype: int64


In [33]:
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] == 1.0) & 
            (moaks_OA_features['PartialTCL'] == 1.0) & 
            (moaks_OA_features['BML'] == 1.0)),'Osteo_AND_PartialTCL_AND_BML'] = 1.0
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] != 1.0) | 
            (moaks_OA_features['PartialTCL'] != 1.0) | 
            (moaks_OA_features['BML'] != 1.0)),'Osteo_AND_PartialTCL_AND_BML'] = 0.0
print('With Definite Osteophytes AND PartialTCL AND BML->1.0, Without->0.0 \n{}'.
        format(moaks_OA_criteria.Osteo_AND_PartialTCL_AND_BML.value_counts(dropna=False)))

With Definite Osteophytes AND PartialTCL AND BML->1.0, Without->0.0 
0.0    54
1.0    43
Name: Osteo_AND_PartialTCL_AND_BML, dtype: int64


In [34]:
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] == 1.0) & 
            (moaks_OA_features['PartialTCL'] == 1.0) & 
            (moaks_OA_features['Meniscus_Degradation'] == 1.0)),'Osteo_AND_PartialTCL_AND_Meniscus_Degradation'] = 1.0
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] != 1.0) | 
            (moaks_OA_features['PartialTCL'] != 1.0) | 
            (moaks_OA_features['Meniscus_Degradation'] != 1.0)),'Osteo_AND_PartialTCL_AND_Meniscus_Degradation'] = 0.0
print('With Osteophytes AND Partial TCL AND Meniscus_Degradation->1.0, Without->0.0: \n{}'.
        format((moaks_OA_criteria['Osteo_AND_PartialTCL_AND_Meniscus_Degradation'].value_counts(dropna=False)))) 

With Osteophytes AND Partial TCL AND Meniscus_Degradation->1.0, Without->0.0: 
1.0    67
0.0    30
Name: Osteo_AND_PartialTCL_AND_Meniscus_Degradation, dtype: int64


In [35]:
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] == 1.0) & 
            (moaks_OA_features['BML'] == 1.0) & 
            (moaks_OA_features['Meniscus_Degradation'] == 1.0)),'Osteo_AND_BML_AND_Meniscus_Degradation'] = 1.0
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] != 1.0) |
            (moaks_OA_features['BML'] != 1.0) |
            (moaks_OA_features['Meniscus_Degradation'] != 1.0)),'Osteo_AND_BML_AND_Meniscus_Degradation'] = 0.0            
print('With Osteophytes AND BML AND Meniscus_Degradation->1.0, Without->0.0: \n{}'.
        format((moaks_OA_criteria['Osteo_AND_BML_AND_Meniscus_Degradation'].value_counts(dropna=False)))) 

With Osteophytes AND BML AND Meniscus_Degradation->1.0, Without->0.0: 
0.0    54
1.0    43
Name: Osteo_AND_BML_AND_Meniscus_Degradation, dtype: int64


In [36]:
moaks_OA_criteria.loc[((moaks_OA_features['FullTCL'] == 1.0) & 
            (moaks_OA_features['BML'] == 1.0) & 
            (moaks_OA_features['Meniscus_Degradation'] == 1.0)),'FullTCL_AND_BML_AND_Meniscus_Degradation'] = 1.0
moaks_OA_criteria.loc[((moaks_OA_features['FullTCL'] != 1.0) | 
            (moaks_OA_features['BML'] != 1.0) | 
            (moaks_OA_features['Meniscus_Degradation'] == 0.0)),'FullTCL_AND_BML_AND_Meniscus_Degradation'] = 0.0
print('With FullTCL AND BML AND Meniscus_Degradation->1.0, Without->0.0: \n{}'.
        format((moaks_OA_criteria['FullTCL_AND_BML_AND_Meniscus_Degradation'].value_counts(dropna=False)))) 


With FullTCL AND BML AND Meniscus_Degradation->1.0, Without->0.0: 
0.0    93
NaN     2
1.0     2
Name: FullTCL_AND_BML_AND_Meniscus_Degradation, dtype: int64


In [37]:
print('Number of subjects without OA for each criterion is: \n{}'.format(moaks_OA_criteria.isna().sum()))

Number of subjects without OA for each criterion is: 
ID                                               0
SIDE                                             0
Osteo_AND_FullTCL                                0
Osteo_AND_PartialTCL_AND_BML                     0
Osteo_AND_PartialTCL_AND_Meniscus_Degradation    0
Osteo_AND_BML_AND_Meniscus_Degradation           0
FullTCL_AND_BML_AND_Meniscus_Degradation         2
dtype: int64


In [38]:
moaks_OA_criteria.fillna(-1.0,inplace=True)

In [39]:
moaks_OA_criteria.to_csv('moaks_sq01_OA_criteria.csv')

In [40]:
# NOW WE FIND THOSE WHO SATISFY AT LEAST ONE OF THE OA CRITERIA
moaks_OA_criteria['moaks_OA'] = np.nan
moaks_OA_criteria.loc[((moaks_OA_criteria['Osteo_AND_FullTCL']==1.0) |
                (moaks_OA_criteria['Osteo_AND_PartialTCL_AND_BML']==1.0) |
                (moaks_OA_criteria['Osteo_AND_PartialTCL_AND_Meniscus_Degradation']==1.0) |
                (moaks_OA_criteria['Osteo_AND_BML_AND_Meniscus_Degradation']==1.0) |
                (moaks_OA_criteria['FullTCL_AND_BML_AND_Meniscus_Degradation']==1.0)),'moaks_OA'] = 1.0
# moaks_OA_criteria.loc[((moaks_OA_criteria['Osteo_AND_FullTCL']==0.0) &
#                 (moaks_OA_criteria['Osteo_AND_PartialTCL_AND_BML']==0.0) &
#                 (moaks_OA_criteria['Osteo_AND_PartialTCL_AND_Meniscus_Degradation']==0.0) &
#                 (moaks_OA_criteria['Osteo_AND_BML_AND_Meniscus_Degradation']==0.0) &
#                 (moaks_OA_criteria['FullTCL_AND_BML_AND_Meniscus_Degra`dation']==0.0)),'moaks_OA'] = 0.0
moaks_OA_criteria.fillna(0.0,inplace=True)

print('Number of subjects that satisfy at least one OA criterion is : {}'.format((moaks_OA_criteria['moaks_OA']==1.0).sum()))

Number of subjects that satisfy at least one OA criterion is : 81


In [41]:
print(moaks_OA_criteria.shape[0],'\n',moaks_OA_criteria['moaks_OA'].value_counts())

97 
 1.0    81
0.0    16
Name: moaks_OA, dtype: int64


In [42]:
print(moaks_OA_criteria)

         ID  SIDE  Osteo_AND_FullTCL  Osteo_AND_PartialTCL_AND_BML  \
0   9002316     2                0.0                           1.0   
1   9026695     1                0.0                           1.0   
2   9044788     2                0.0                           1.0   
3   9062483     2                0.0                           0.0   
4   9063928     2                0.0                           1.0   
..      ...   ...                ...                           ...   
92  9943227     2                0.0                           1.0   
93  9957519     2                0.0                           0.0   
94  9961728     2                0.0                           0.0   
95  9984706     1                0.0                           1.0   
96  9988820     2                0.0                           0.0   

    Osteo_AND_PartialTCL_AND_Meniscus_Degradation  \
0                                             1.0   
1                                             1.0   


# NOW WE CREATE THE DATAFRAME CONTAINING INFORMATION REGARDING 
# 1.MOAKS FEATURES 
# 2.MOAKS CRITERIA
# 3.MOAKS GRADES
# 4.KL GRADES
# 5.MOAKS-KL GRADES CROSSTABLE

In [43]:
# Here we upload the DataFrame with the KL grades
# and we merge the MOAKS and KL dataframes
kl_id_side_grade = pd.read_csv('/home/anastasis/EMC_Thesis/My_codes/KL_readings/KL_SQ01_ID_SIDE_GRADE.csv')
moaks_kl_OA_full = moaks_OA_criteria.merge(kl_id_side_grade,on=['ID','SIDE'],how='inner')
print(moaks_kl_OA_full.shape[0],'\n',moaks_kl_OA_full)

97 
          ID  SIDE  Osteo_AND_FullTCL  Osteo_AND_PartialTCL_AND_BML  \
0   9002316     2                0.0                           1.0   
1   9026695     1                0.0                           1.0   
2   9044788     2                0.0                           1.0   
3   9062483     2                0.0                           0.0   
4   9063928     2                0.0                           1.0   
..      ...   ...                ...                           ...   
92  9943227     2                0.0                           1.0   
93  9957519     2                0.0                           0.0   
94  9961728     2                0.0                           0.0   
95  9984706     1                0.0                           1.0   
96  9988820     2                0.0                           0.0   

    Osteo_AND_PartialTCL_AND_Meniscus_Degradation  \
0                                             1.0   
1                                             1.

In [44]:
print('Number of different KL grades in the moaks_kl_OA_full dataframe:\n{}'
        .format(moaks_kl_OA_full['V01XRKL'].value_counts()))
moaks_kl_OA_full.to_csv('moaks_sq01_OA_KL_both_grades.csv')

Number of different KL grades in the moaks_kl_OA_full dataframe:
2.0    78
3.0    18
4.0     1
Name: V01XRKL, dtype: int64


In [45]:
moaks_KL_OA_all_info = moaks_OA_features.merge(moaks_kl_OA_full,on=['ID','SIDE'],how='outer',indicator=True)
print(moaks_KL_OA_all_info._merge.value_counts(),'\n',moaks_KL_OA_all_info.info())
moaks_KL_OA_all_info.to_csv('moaks_sq01_all_info.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Data columns (total 16 columns):
 #   Column                                         Non-Null Count  Dtype   
---  ------                                         --------------  -----   
 0   ID                                             97 non-null     int64   
 1   SIDE                                           97 non-null     int64   
 2   XR_Osteophytes                                 97 non-null     float64 
 3   FullTCL                                        97 non-null     float64 
 4   AnyTCL                                         97 non-null     float64 
 5   PartialTCL                                     97 non-null     float64 
 6   BML                                            97 non-null     float64 
 7   Meniscus_Degradation                           97 non-null     float64 
 8   Osteo_AND_FullTCL                              97 non-null     float64 
 9   Osteo_AND_PartialTCL_AND_BML                 

In [46]:
all_common_sq_kxr_01_sq01 = moaks_KL_OA_all_info.loc[:,['ID','SIDE','moaks_OA']]
print(all_common_sq_kxr_01_sq01.info())
all_moaks_vs_kl_grades_sq01 = moaks_KL_OA_all_info.loc[:,['ID','SIDE','moaks_OA','V01XRKL']]
print(all_moaks_vs_kl_grades_sq01.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        97 non-null     int64  
 1   SIDE      97 non-null     int64  
 2   moaks_OA  97 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 3.0 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        97 non-null     int64  
 1   SIDE      97 non-null     int64  
 2   moaks_OA  97 non-null     float64
 3   V01XRKL   97 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 3.8 KB
None


In [47]:
print('Sum of sq01 subjects that have MOAKS = 0.0: {}'.format((all_common_sq_kxr_01_sq01['moaks_OA']==0.0).sum()))
print('Sum of sq01 subjects that have MOAKS = 1.0: {}'.format((all_common_sq_kxr_01_sq01['moaks_OA']==1.0).sum()))

Sum of sq01 subjects that have MOAKS = 0.0: 16
Sum of sq01 subjects that have MOAKS = 1.0: 81


In [48]:
moaks_sq01_only_moaks_one = all_moaks_vs_kl_grades_sq01.loc[(all_moaks_vs_kl_grades_sq01['moaks_OA']==1.0)]
print(moaks_sq01_only_moaks_one,'\n','number of subjects with moaks grade = 1.0: ',moaks_sq01_only_moaks_one.moaks_OA.count())

         ID  SIDE  moaks_OA  V01XRKL
0   9002316     2       1.0      2.0
1   9026695     1       1.0      3.0
2   9044788     2       1.0      2.0
4   9063928     2       1.0      2.0
5   9067323     2       1.0      2.0
..      ...   ...       ...      ...
91  9916319     2       1.0      2.0
92  9943227     2       1.0      2.0
93  9957519     2       1.0      2.0
95  9984706     1       1.0      2.0
96  9988820     2       1.0      2.0

[81 rows x 4 columns] 
 number of subjects with moaks grade = 1.0:  81


In [49]:
moaks_sq01_only_moaks_zero = all_moaks_vs_kl_grades_sq01.loc[(all_moaks_vs_kl_grades_sq01['moaks_OA']==0.0)]
print(moaks_sq01_only_moaks_zero,'\n','number of subjects with moaks grade = 0.0: ',moaks_sq01_only_moaks_zero.moaks_OA.count())

         ID  SIDE  moaks_OA  V01XRKL
3   9062483     2       0.0      2.0
9   9101270     2       0.0      2.0
14  9169022     1       0.0      2.0
20  9232259     1       0.0      2.0
22  9268233     1       0.0      2.0
24  9272198     1       0.0      2.0
42  9487864     1       0.0      2.0
44  9504056     2       0.0      2.0
50  9568180     2       0.0      2.0
54  9599243     1       0.0      2.0
55  9609732     1       0.0      2.0
65  9716478     2       0.0      2.0
80  9841815     2       0.0      2.0
81  9860552     1       0.0      3.0
83  9883539     2       0.0      2.0
94  9961728     2       0.0      2.0 
 number of subjects with moaks grade = 0.0:  16


In [50]:
# Here we save the dataframes containing the information we want
all_moaks_vs_kl_grades_sq01.to_csv('moaks_sq01_moaks_vs_kl.csv')
moaks_sq01_only_moaks_one.to_csv('moaks_sq01_only_moaks_one.csv')
moaks_sq01_only_moaks_zero.to_csv('moaks_sq01_only_moaks_zero.csv')
all_common_sq_kxr_01_sq01.to_csv('moaks_sq01_common_sq_kxr_01.csv')

In [51]:
moaks_OA_KL_crosstab = pd.crosstab(all_moaks_vs_kl_grades_sq01.moaks_OA,all_moaks_vs_kl_grades_sq01.V01XRKL,margins=True,margins_name='Total')
moaks_OA_KL_crosstab.to_csv('moaks_sq01_OA_KL_crosstab.csv')
moaks_OA_KL_crosstab


V01XRKL,2.0,3.0,4.0,Total
moaks_OA,,,,
0.0,15,1,0,16
1.0,63,17,1,81
Total,78,18,1,97


In [52]:
# NOW WE SAVE THE SUBJECTS WITH ZERO FEATURES IN ORDER TO INVESTIGATE THE INCIDENCE OA
moaks_OA_osteo_zero = moaks_OA_features.loc[moaks_OA_features.XR_Osteophytes == 0.0]
moaks_OA_osteo_zero.to_csv('moaks_sq01_Zero_Osteo.csv')
print('Number of subjects without definite osteophytes in SQ01 is : {}'.format(moaks_OA_osteo_zero.shape[0]))

moaks_OA_TCL_zero = moaks_OA_features.loc[(moaks_OA_features.FullTCL == 0.0) & (moaks_OA_features.PartialTCL == 0.0)]
moaks_OA_TCL_zero.to_csv('moaks_sq01_Zero_TCL.csv')
print('Number of subjects without TCL in SQ01 is : {}'.format(moaks_OA_TCL_zero.shape[0]))

moaks_OA_bml_zero = moaks_OA_features.loc[moaks_OA_features.BML == 0.0]
moaks_OA_bml_zero.to_csv('moaks_sq01_Zero_BML.csv')
print('Number of subjects without BML in SQ01 is : {}'.format(moaks_OA_bml_zero.shape[0]))

moaks_OA_meniscus_zero = moaks_OA_features.loc[moaks_OA_features.Meniscus_Degradation == 0.0]
moaks_OA_meniscus_zero.to_csv('moaks_sq01_Zero_Meniscus.csv')
print('Number of subjects without Meniscus Degradation in SQ01 is : {}'.format(moaks_OA_meniscus_zero.shape[0]))


Number of subjects without definite osteophytes in SQ01 is : 0
Number of subjects without TCL in SQ01 is : 9
Number of subjects without BML in SQ01 is : 43
Number of subjects without Meniscus Degradation in SQ01 is : 0
